1. Top-n approach: In this approach the best "n" number of restaurents are recommended
2. Value-based approach: In this appproach the recommended restaurents are above/below a particular threshold value.

#### Reading the dataset

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [4]:
data = pd.read_csv('/Users/hackthebox/Downloads/Machine_Learning_for_Text_And_Image_Data_Analysis/Section-2_Unsupervised_Machine_Learning_for_Text_And_Image_Data_Analysis/Content_Based_Recommendation_System/Content_Based_Recommendation_Text_Data/Data/restaurant.csv',encoding='ISO-8859-1')
print("Dimension of dataset:",data.shape)

Dimension of dataset: (250, 3)


In [5]:
data.head()

,Restaurant,City,Cuisine
0,Kochi Kitchen,Kerala,"North Indian, Mughlai, Chinese"
1,Hotel Chandra,Hyderabad,"Cafe, Mexican, Italian"
2,Cafe Coffee Day Surat,Surat,"North Indian, Rajasthani"
3,Hotel KRC Palace,Tezpur,North Indian
4,Galaxy Surat,Surat,"North Indian, South Indian, Andhra, Chinese"


In [6]:
data.tail()

,Restaurant,City,Cuisine
245,Swagath Restaurant,Gurgaon,"Mughlai, South Indian"
246,MKT,New Delhi,"Italian, Mexican, Indian"
247,Thyme,New Delhi,"Indian, International"
248,Food Exchange,New Delhi,Indian
249,Prego,Gurgaon,Italian


In [7]:
# Creating one copy of the dataset for later reference
fooddata = data.copy()

In [8]:
# Installing geopy library for determining latitude and longitude
!pip install geopy

#### Creating location column for determining longitude and latitude

In [10]:
data["Location"] = data["Restaurant"] + "," + data["City"]
data.head()

,Restaurant,City,Cuisine,Location
0,Kochi Kitchen,Kerala,"North Indian, Mughlai, Chinese","Kochi Kitchen,Kerala"
1,Hotel Chandra,Hyderabad,"Cafe, Mexican, Italian","Hotel Chandra,Hyderabad"
2,Cafe Coffee Day Surat,Surat,"North Indian, Rajasthani","Cafe Coffee Day Surat,Surat"
3,Hotel KRC Palace,Tezpur,North Indian,"Hotel KRC Palace,Tezpur"
4,Galaxy Surat,Surat,"North Indian, South Indian, Andhra, Chinese","Galaxy Surat,Surat"


#### Creating Nomination object from geopy library

In [19]:
from geopy.geocoders import Nominatim
geocoder = Nominatim(user_agent='xxx')

#### Creating object for rate limiter from geopy library

In [23]:
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geocoder.geocode,min_delay_seconds=1,return_value_on_exception=None)

#### Determining latitude and longituden from the location 

In [52]:
# Creating latitude and longitude lists

latlist = []
lonlist = []

for i in range(250):
    location = geocode(data['Location'][i])
    if location is not None:
        latlist.append(location.latitude)
        lonlist.append(location.longitude)
    else:
        latlist.append(None)  # or a default value, e.g., 0
        lonlist.append(None)  # or a default value, e.g., 0

# Creating new columns named latitude and longitude from lists
data['Latitude'] = latlist
data['Longitude'] = lonlist


In [53]:
data.head()

,Restaurant,City,Cuisine,Location,Latitude,Longitude,North,South,Desserts,Continental,Italian,Indian
0,Kochi Kitchen,Kerala,"North Indian, Mughlai, Chinese","Kochi Kitchen,Kerala",NaN,NaN,True,False,False,False,False,True
1,Hotel Chandra,Hyderabad,"Cafe, Mexican, Italian","Hotel Chandra,Hyderabad",NaN,NaN,False,False,False,False,True,False
2,Cafe Coffee Day Surat,Surat,"North Indian, Rajasthani","Cafe Coffee Day Surat,Surat",21.171268,72.789414,True,False,False,False,False,True
3,Hotel KRC Palace,Tezpur,North Indian,"Hotel KRC Palace,Tezpur",26.626499,92.790122,True,False,False,False,False,True
4,Galaxy Surat,Surat,"North Indian, South Indian, Andhra, Chinese","Galaxy Surat,Surat",21.200953,72.835225,True,True,False,False,False,True


In [63]:
# data['Latitude'].interpolate(method='linear', inplace=True)
# data['Longitude'].interpolate(method='linear', inplace=True)

In [66]:
data['Latitude'].fillna(data['Latitude'].mean(), inplace=True)
data['Longitude'].fillna(data['Longitude'].mean(), inplace=True)


In [67]:
data.dropna()

,Restaurant,City,Cuisine,Location,Latitude,Longitude,North,South,Desserts,Continental,Italian,Indian
0,Kochi Kitchen,Kerala,"North Indian, Mughlai, Chinese","Kochi Kitchen,Kerala",21.919974,73.763960,True,False,False,False,False,True
1,Hotel Chandra,Hyderabad,"Cafe, Mexican, Italian","Hotel Chandra,Hyderabad",21.919974,73.763960,False,False,False,False,True,False
2,Cafe Coffee Day Surat,Surat,"North Indian, Rajasthani","Cafe Coffee Day Surat,Surat",21.171268,72.789414,True,False,False,False,False,True
3,Hotel KRC Palace,Tezpur,North Indian,"Hotel KRC Palace,Tezpur",26.626499,92.790122,True,False,False,False,False,True
4,Galaxy Surat,Surat,"North Indian, South Indian, Andhra, Chinese","Galaxy Surat,Surat",21.200953,72.835225,True,True,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
245,Swagath Restaurant,Gurgaon,"Mughlai, South Indian","Swagath Restaurant,Gurgaon",28.469818,77.062759,False,True,False,False,False,True
246,MKT,New Delhi,"Italian, Mexican, Indian","MKT,New Delhi",37.785468,-122.409082,False,False,False,False,True,True
247,Thyme,New Delhi,"Indian, International","Thyme,New Delhi",37.785468,-122.409082,False,False,False,False,False,True
248,Food Exchange,New Delhi,Indian,"Food Exchange,New Delhi",37.785468,-122.409082,False,False,False,False,False,True


In [68]:
data.isnull().sum()

Restaurant     0
City           0
Cuisine        0
Location       0
Latitude       0
Longitude      0
North          0
South          0
Desserts       0
Continental    0
Italian        0
Indian         0
dtype: int64

#### Creating new columns of Boolean values depending on cuisines

In [69]:
data['North'] = data['Cuisine'].astype(str).apply(lambda x:"North" in x)
data['South'] = data['Cuisine'].astype(str).apply(lambda x:"South" in x)
data['Desserts'] = data['Cuisine'].astype(str).apply(lambda x:"Desserts" in x)
data['Continental'] = data['Cuisine'].astype(str).apply(lambda x:"Continental" in x)
data['Italian'] = data['Cuisine'].astype(str).apply(lambda x:"Italian" in x)
data['Indian'] = data['Cuisine'].astype(str).apply(lambda x:"Indian" in x)


In [70]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250 entries, 0 to 249
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Restaurant   250 non-null    object 
 1   City         250 non-null    object 
 2   Cuisine      250 non-null    object 
 3   Location     250 non-null    object 
 4   Latitude     250 non-null    float64
 5   Longitude    250 non-null    float64
 6   North        250 non-null    bool   
 7   South        250 non-null    bool   
 8   Desserts     250 non-null    bool   
 9   Continental  250 non-null    bool   
 10  Italian      250 non-null    bool   
 11  Indian       250 non-null    bool   
dtypes: bool(6), float64(2), object(4)
memory usage: 13.3+ KB


In [71]:
data.head()

,Restaurant,City,Cuisine,Location,Latitude,Longitude,North,South,Desserts,Continental,Italian,Indian
0,Kochi Kitchen,Kerala,"North Indian, Mughlai, Chinese","Kochi Kitchen,Kerala",21.919974,73.763960,True,False,False,False,False,True
1,Hotel Chandra,Hyderabad,"Cafe, Mexican, Italian","Hotel Chandra,Hyderabad",21.919974,73.763960,False,False,False,False,True,False
2,Cafe Coffee Day Surat,Surat,"North Indian, Rajasthani","Cafe Coffee Day Surat,Surat",21.171268,72.789414,True,False,False,False,False,True
3,Hotel KRC Palace,Tezpur,North Indian,"Hotel KRC Palace,Tezpur",26.626499,92.790122,True,False,False,False,False,True
4,Galaxy Surat,Surat,"North Indian, South Indian, Andhra, Chinese","Galaxy Surat,Surat",21.200953,72.835225,True,True,False,False,False,True


- It should be noted that the distences cannot be applied on the Boolean values or Categorical variables.

In [72]:
# Removing city,location and Cuisine columns from the dataset
finaldata = data.drop(columns=['City', 'Location', 'Cuisine'])

In [73]:
# Converting Boolean to integer value
finaldata = finaldata * 1

In [74]:
# Creating the index as the name of the restaurent
finaldata.set_index("Restaurant",inplace=True)

In [75]:
finaldata.head()

,Latitude,Longitude,North,South,Desserts,Continental,Italian,Indian
Restaurant,,,,,,,,
Kochi Kitchen,21.919974,73.763960,1,0,0,0,0,1
Hotel Chandra,21.919974,73.763960,0,0,0,0,1,0
Cafe Coffee Day Surat,21.171268,72.789414,1,0,0,0,0,1
Hotel KRC Palace,26.626499,92.790122,1,0,0,0,0,1
Galaxy Surat,21.200953,72.835225,1,1,0,0,0,1


# COSINE SIMILARITY FOR TEXT DATA ANALYSIS

In [76]:
# Determining criterio for search of restaurent
place = input("Enter the name of the Restaurent:")
# Determining the details of the place from the dataset
print("Details of Restaurent:",fooddata[fooddata["Restaurant"]==place])
# Determining the index of the place
search = fooddata.index[fooddata['Restaurant']==place]
item = search[0]
print("The Restaurent is located at index:",item)

Details of Restaurent:     Restaurant       City Cuisine
240    Dakshin  Bangalore  Indian
The Restaurent is located at index: 240


In [77]:
# Recommendation system using cosine similarity
# Importing necessary libraries
from sklearn.metrics.pairwise import cosine_similarity

In [78]:
# Determining cosine similarity
sim_cossim = cosine_similarity(finaldata)
df_cossim = pd.DataFrame(sim_cossim)
df_cossim.head()

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
0,1.000000,0.999747,0.999983,0.999949,0.999897,0.998696,0.999669,0.999896,0.998692,0.993636,...,0.992445,0.992333,0.995828,0.998205,0.999540,0.997728,-0.831618,-0.831643,-0.831643,-0.831744
1,0.999747,1.000000,0.999725,0.999764,0.999639,0.998938,0.999918,0.999638,0.998934,0.993879,...,0.992363,0.992252,0.995749,0.998128,0.999465,0.997654,-0.831688,-0.831814,-0.831814,-0.831713
2,0.999983,0.999725,1.000000,0.999985,0.999913,0.998407,0.999634,0.999913,0.998403,0.992973,...,0.993133,0.993027,0.996333,0.998525,0.999679,0.997331,-0.834818,-0.834844,-0.834844,-0.834947
3,0.999949,0.999764,0.999985,1.000000,0.999898,0.998280,0.999664,0.999899,0.998276,0.992614,...,0.993576,0.993473,0.996662,0.998740,0.999783,0.997102,-0.836872,-0.836897,-0.836897,-0.836978
4,0.999897,0.999639,0.999913,0.999898,1.000000,0.998331,0.999549,1.000000,0.998327,0.992911,...,0.993023,0.992917,0.996229,0.998427,0.999588,0.997420,-0.834632,-0.834658,-0.834658,-0.834761


In [79]:
# Determining the similar restaurents to the given restaurent
sim_cossim_rest = df_cossim.iloc[item].values
print("Cosine Similarity - Restaurent Similarities:\n",sim_cossim_rest)

Cosine Similarity - Restaurent Similarities:
 [ 0.99244452  0.99236304  0.99313342  0.99357586  0.9930232   0.98569829
  0.9919988   0.9930476   0.98568513  0.9728185   0.99908275  0.99350101
  0.99896653  0.99869647  0.99196735  0.98778558  0.97260676  0.98578706
  0.99317378  0.98568828  0.99857977  0.99315861  0.98423133  0.97260663
  0.97977363  0.98569079  0.99600013  0.99201118  0.98579052  0.99973329
  0.99078358  0.99907385  0.99901277  0.99078919  0.99207006  0.99072205
  0.98576892  0.98567437  0.9908544   0.99313351  0.98570017  0.99211943
  0.99885134  0.99600566  0.99986819  0.97281439  0.9990709   0.99895689
  0.99083426  0.99202876  0.98475189  0.99071348  0.99896271  0.98580864
  0.98484332  0.9921575   0.97288308  0.99083732  0.97280074  0.99904103
  0.99981651  0.9985134   0.9821572   0.99081693  0.99139542  0.99214831
  0.99857039  0.99989218  0.99318087  0.99916211  0.99905926  0.99767256
  0.98576856  0.99822753  0.99214443  0.99219133  0.98577675  0.99858346
  0.9

In [80]:
# Top-3 approach for recommendation of 3 similar restaurents
# Displaying index of first 3 similar restaurents with restaurent index 100
sim_cossim_idx = np.argsort(-sim_cossim_rest)[0:3]
print("Cosine Similarity - Index of similar restaurents:\n",sim_cossim_idx)

Cosine Similarity - Index of similar restaurents:
 [240 241 237]


In [81]:
# Similarity with the 3 similar restaurents
for i in range(3):
    print("Cosine Similarity:",sim_cossim_rest[i])
    print("Restaurant Name:", fooddata.iloc[sim_cossim_idx[i]].Restaurant)
    print("Details:", fooddata.iloc[sim_cossim_idx[i]])
    print("-----------------------------------------------")

Cosine Similarity: 0.9924445241640732
Restaurant Name: Dakshin
Details: Restaurant      Dakshin
City          Bangalore
Cuisine          Indian
Name: 240, dtype: object
-----------------------------------------------
Cosine Similarity: 0.9923630449802021
Restaurant Name: Karavalli
Details: Restaurant          Karavalli
City                Bangalore
Cuisine       Indian, Seafood
Name: 241, dtype: object
-----------------------------------------------
Cosine Similarity: 0.9931334211048517
Restaurant Name: Sangeetha Veg Restaurant
Details: Restaurant    Sangeetha Veg Restaurant
City                           Chennai
Cuisine                         Indian
Name: 237, dtype: object
-----------------------------------------------


In [82]:
# Similarity with the 3 similar restaurents
for i in range(3):
    print('Similarity score with restaurent at index',sim_cossim_idx[i],'is:',sim_cossim_rest[sim_cossim_idx[i]].round(2))
    print("Details of restaurent:\n",fooddata.values[sim_cossim_idx[i],:])


Similarity score with restaurent at index 240 is: 1.0
Details of restaurent:
 ['Dakshin' 'Bangalore' 'Indian']
Similarity score with restaurent at index 241 is: 1.0
Details of restaurent:
 ['Karavalli' 'Bangalore' 'Indian, Seafood']
Similarity score with restaurent at index 237 is: 1.0
Details of restaurent:
 ['Sangeetha Veg Restaurant' 'Chennai' 'Indian']


In [83]:
# Value-based approach for recommendation of similar restaurents
value_idx = np.where(sim_cossim_rest>0.999)
print("Number of restaurents greater than 0.9999:",len(sum(value_idx)))
print("Index of similar restaurents based on value:\n",value_idx)

Number of restaurents greater than 0.9999: 32
Index of similar restaurents based on value:
 (array([ 10,  29,  31,  32,  44,  46,  59,  60,  67,  69,  70,  84, 115,
       116, 128, 141, 162, 179, 187, 188, 198, 200, 229, 230, 231, 235,
       236, 237, 239, 240, 241, 242]),)


# COSINE DISTANCE FOR TEXT DATA ANALYSIS

In [84]:
# Determining criterio for search of restaurent
place = input("Enter the name of the Restaurent:")
# Determining the details of the place from the dataset
print("Details of Restaurent:",fooddata[fooddata["Restaurant"]==place])
# Determining the index of the place
search = fooddata.index[fooddata['Restaurant']==place]
item = search[0]
print("The Restaurent is located at index:",item)

Details of Restaurent:     Restaurant       City                   Cuisine
246        MKT  New Delhi  Italian, Mexican, Indian
The Restaurent is located at index: 246


In [85]:
# Recommendation system using cosine distance
# Importing necessary libraries
from sklearn.metrics.pairwise import cosine_distances

In [86]:
# Determining cosine distance
sim_cossine = cosine_distances(finaldata)
df_cosine = pd.DataFrame(sim_cossine)
df_cosine.head()

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
0,0.000000,0.000253,0.000017,0.000051,0.000103,0.001304,0.000331,1.038498e-04,0.001308,0.006364,...,0.007555,0.007667,0.004172,0.001795,0.000460,0.002272,1.831618,1.831643,1.831643,1.831744
1,0.000253,0.000000,0.000275,0.000236,0.000361,0.001062,0.000082,3.621248e-04,0.001066,0.006121,...,0.007637,0.007748,0.004251,0.001872,0.000535,0.002346,1.831688,1.831814,1.831814,1.831713
2,0.000017,0.000275,0.000000,0.000015,0.000087,0.001593,0.000366,8.694250e-05,0.001597,0.007027,...,0.006867,0.006973,0.003667,0.001475,0.000321,0.002669,1.834818,1.834844,1.834844,1.834947
3,0.000051,0.000236,0.000015,0.000000,0.000102,0.001720,0.000336,1.014189e-04,0.001724,0.007386,...,0.006424,0.006527,0.003338,0.001260,0.000217,0.002898,1.836872,1.836897,1.836897,1.836978
4,0.000103,0.000361,0.000087,0.000102,0.000000,0.001669,0.000451,2.240963e-08,0.001673,0.007089,...,0.006977,0.007083,0.003771,0.001573,0.000412,0.002580,1.834632,1.834658,1.834658,1.834761


In [87]:
# Determining the similar restaurents to the given restaurent
sim_cosine_rest = df_cosine.iloc[item].values
print("Cosine Distance - Restaurent Similarities:\n",sim_cosine_rest)

Cosine Distance - Restaurent Similarities:
 [1.83161751e+00 1.83168771e+00 1.83481842e+00 1.83687168e+00
 1.83463230e+00 1.80525454e+00 1.83031979e+00 1.83474827e+00
 1.80520806e+00 1.76531268e+00 1.90891023e+00 1.83701484e+00
 1.91111817e+00 1.87063044e+00 1.82984478e+00 1.81288095e+00
 1.76487057e+00 1.80522051e+00 1.83523395e+00 1.80521919e+00
 1.86891158e+00 1.83493794e+00 1.80007352e+00 1.76443522e+00
 1.78539290e+00 1.80532271e+00 1.84990099e+00 1.82990097e+00
 1.80547895e+00 1.90149295e+00 1.82457114e+00 1.91077947e+00
 1.91166415e+00 1.82438899e+00 1.83035141e+00 1.82444830e+00
 1.80515641e+00 1.80506870e+00 1.82464151e+00 1.83481885e+00
 1.80535568e+00 1.83042957e+00 1.91135543e+00 1.85023086e+00
 1.88877184e+00 1.76525016e+00 1.91109297e+00 1.91219460e+00
 1.82455688e+00 1.82997915e+00 1.80161320e+00 1.82441226e+00
 1.91125691e+00 1.80529688e+00 1.80216973e+00 1.83031186e+00
 1.76520391e+00 1.82456968e+00 1.76535921e+00 1.91110461e+00
 1.89029666e+00 1.86801857e+00 1.79312107

In [88]:
# Top-3 approach for recommendation of 3 similar restaurents
# Displaying index of first 3 similar restaurents with restaurent index 204
sim_cosine_idx = np.argsort(-sim_cosine_rest)[0:3]
print("Cosine Distance - Index of similar restaurents:\n",sim_cosine_idx)

Cosine Distance - Index of similar restaurents:
 [ 73  47 109]


In [89]:
# Distances with the 3 similar restaurents
for i in range(3):
    print("Cosine Distance:",sim_cosine_rest[i])
    print("Restaurant Name:", fooddata.iloc[sim_cosine_idx[i]].Restaurant)
    print("Details:", fooddata.iloc[sim_cosine_idx[i]])
    print("-----------------------------------------------")

Cosine Similarity: 1.8316175072907446
Restaurant Name: Villa Maya
Details: Restaurant          Villa Maya
City                    Kerala
Cuisine       Bakery, Desserts
Name: 73, dtype: object
-----------------------------------------------
Cosine Similarity: 1.8316877113749404
Restaurant Name: Arabian Nights Restaurant
Details: Restaurant    Arabian Nights Restaurant
City                           Kottayam
Cuisine                         Chinese
Name: 47, dtype: object
-----------------------------------------------
Cosine Similarity: 1.8348184242395542
Restaurant Name: Taste of Kottayam
Details: Restaurant    Taste of Kottayam
City                   Kottayam
Cuisine            North Indian
Name: 109, dtype: object
-----------------------------------------------


In [90]:
# Distances with the 3 similar restaurents
for i in range(3):
    print('\nDistance  with restaurent at index',sim_cosine_idx[i],'is:',sim_cosine_rest[sim_cosine_idx[i]].round(2))
    print("Details of restaurent:\n",fooddata.values[sim_cosine_idx[i],:])


Distance  with restaurent at index 73 is: 1.92
Details of restaurent:
 ['Villa Maya' 'Kerala' 'Bakery, Desserts']

Distance  with restaurent at index 47 is: 1.91
Details of restaurent:
 ['Arabian Nights Restaurant' 'Kottayam' 'Chinese']

Distance  with restaurent at index 109 is: 1.91
Details of restaurent:
 ['Taste of Kottayam' 'Kottayam' 'North Indian']


In [91]:
# Value-based approach for recommendation of similar restaurents
value_idx = np.where(sim_cosine_rest<0.001)
print("Number of restaurents <0.001 value:",len(sum(value_idx)))
print("Index of similar restaurents based on value:\n",value_idx)

Number of restaurents <0.001 value: 4
Index of similar restaurents based on value:
 (array([246, 247, 248, 249]),)


# EUCLIDEAN DISTANCE FOR TEXT DATA ANALYSIS

In [92]:
# Determining criterio for search of restaurent
place = input("Enter the name of the Restaurent:")
# Determining the details of the place from the dataset
print("Details of Restaurent:",fooddata[fooddata["Restaurant"]==place])
# Determining the index of the place
search = fooddata.index[fooddata['Restaurant']==place]
item = search[0]
print("The Restaurent is located at index:",item)

Details of Restaurent:      Restaurant    City                              Cuisine
113  Lemon Tree  Mumbai  Cafe, Chinese, Continental, Italian
The Restaurent is located at index: 113


In [93]:
# Recommendation system using Euclidean distance
# Importing necessary libraries
from sklearn.metrics.pairwise import euclidean_distances

In [94]:
# Determining cosine distance
sim_euc = euclidean_distances(finaldata)
df_cuc = pd.DataFrame(sim_euc)
df_cuc.head()

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
0,0.000000,1.732051,1.228943,19.599648,1.542576,6.993544,3.043714,1.576607,6.992916,9.915748,...,9.706173,9.790232,7.646464,6.038596,5.462353,7.468771,196.818638,196.816098,196.816098,196.821178
1,1.732051,0.000000,2.123747,19.676032,2.319383,6.775667,2.502838,2.342155,6.775019,9.763301,...,9.757550,9.841171,7.711577,6.120836,5.553135,7.535419,196.816098,196.818638,196.818638,196.813557
2,1.228943,2.123747,0.000000,20.731326,1.001489,8.162900,4.023261,1.000060,8.161991,10.794943,...,9.474104,9.563179,7.688718,6.492673,6.369433,8.574982,195.909378,195.906826,195.906826,195.911930
3,19.599648,19.676032,20.731326,0.000000,20.703489,14.850061,17.441479,20.744510,14.856790,18.673895,...,20.458983,20.442912,18.335377,16.426683,14.793992,15.898043,215.492970,215.490649,215.490649,215.495290
4,1.542576,2.319383,1.001489,20.703489,0.000000,8.172996,4.100074,0.044351,8.172108,10.803794,...,9.529437,9.617827,7.743231,6.541338,6.402892,8.468084,195.955060,195.952509,195.952509,195.957612


In [95]:
# Determining the similar restaurents to the given restaurent
sim_euc_rest = df_cuc.iloc[item].values
print("Euclidean Distance - Restaurent Similarities:\n",sim_euc_rest)

Euclidean Distance - Restaurent Similarities:
 [3.67705797e+00 3.24357138e+00 2.96932914e+00 2.14628503e+01
 3.15370197e+00 9.76028827e+00 4.86193128e+00 3.13497259e+00
 9.76058469e+00 1.28452767e+01 9.63276999e+00 2.13987759e+01
 1.01246107e+01 8.17885854e+00 1.40188915e+01 5.74922685e+00
 1.28199249e+01 9.91157602e+00 2.58244720e+00 9.75799884e+00
 6.01199813e+00 2.95049221e+00 7.64089455e+00 1.30367094e+01
 1.12254797e+01 9.82301607e+00 2.02125887e+00 1.40406297e+01
 9.80973364e+00 8.50047939e+00 1.13057960e+01 1.02312242e+01
 1.02963603e+01 5.28670619e+00 4.95630107e+00 1.12070403e+01
 9.90988084e+00 9.81790645e+00 1.12636244e+01 2.96926762e+00
 9.85233012e+00 5.30668302e+00 1.01748839e+01 9.29185748e+00
 7.08261708e+00 1.29117717e+01 1.01419069e+01 1.03309684e+01
 1.12610500e+01 1.40546269e+01 8.69233798e+00 1.12086076e+01
 1.02273560e+01 9.90004149e+00 8.58394220e+00 5.25076584e+00
 1.29688884e+01 1.12879864e+01 1.29295560e+01 1.02789251e+01
 7.33669193e+00 6.08434130e+00 1.08259

In [97]:
# Top-3 approach for recommendation of 3 similar restaurents
# Displaying index of first 3 similar restaurents with restaurent index 165
sim_euc_idx = np.argsort(sim_euc_rest)[0:3]
print("Euclidean Distance - Index of similar restaurents:\n",sim_euc_idx)

Euclidean Distance - Index of similar restaurents:
 [113 175 228]


In [98]:
# Distances with the 3 similar restaurents
for i in range(3):
    print("Cosine Distance:",sim_euc_rest[i])
    print("Restaurant Name:", fooddata.iloc[sim_euc_idx[i]].Restaurant)
    print("Details:", fooddata.iloc[sim_euc_idx[i]])
    print("-----------------------------------------------")

Cosine Distance: 3.6770579683444895
Restaurant Name: Lemon Tree
Details: Restaurant                             Lemon Tree
City                                       Mumbai
Cuisine       Cafe, Chinese, Continental, Italian
Name: 113, dtype: object
-----------------------------------------------
Cosine Distance: 3.24357138083405
Restaurant Name: The Table
Details: Restaurant                     The Table
City                              Mumbai
Cuisine       Cafe, Italian, Continental
Name: 175, dtype: object
-----------------------------------------------
Cosine Distance: 2.969329144195103
Restaurant Name: Four Seasons
Details: Restaurant                          Four Seasons
City                                      Mumbai
Cuisine       Italian, Fast Food, Cafe, European
Name: 228, dtype: object
-----------------------------------------------


In [99]:
# Distances with the 3 similar restaurents
for i in range(3):
    print('\nDistance  with restaurent at index',sim_euc_idx[i],'is:',sim_euc_rest[sim_euc_idx[i]].round(2))
    print("Details of restaurent:\n",fooddata.values[sim_euc_idx[i],:])


Distance  with restaurent at index 113 is: 0.0
Details of restaurent:
 ['Lemon Tree' 'Mumbai' 'Cafe, Chinese, Continental, Italian']

Distance  with restaurent at index 175 is: 0.05
Details of restaurent:
 ['The Table' 'Mumbai' 'Cafe, Italian, Continental']

Distance  with restaurent at index 228 is: 1.0
Details of restaurent:
 ['Four Seasons' 'Mumbai' 'Italian, Fast Food, Cafe, European']


In [101]:
# Value-based approach for recommendation of similar restaurents
value_idx = np.where(sim_euc_rest<1.5)
print("Number of restaurents <1.5 value:",len(sum(value_idx)))
print("Index of similar restaurents based on value:\n",value_idx)

Number of restaurents <1.5 value: 10
Index of similar restaurents based on value:
 (array([ 99, 113, 143, 175, 177, 204, 211, 220, 227, 228]),)


# MANHATTAN DISTANCE FOR TEXT DATA ANALYSIS

In [103]:
# Determining criterio for search of restaurent
place = input("Enter the name of the Restaurent:")
# Determining the details of the place from the dataset
print("Details of Restaurent:",fooddata[fooddata["Restaurant"]==place])
# Determining the index of the place
search = fooddata.index[fooddata['Restaurant']==place]
item = search[0]
print("The Restaurent is located at index:",item)

Details of Restaurent:      Restaurant       City                Cuisine
77  Hotel Abode  Hyderabad  Chinese, North Indian
The Restaurent is located at index: 77


In [104]:
# Recommendation system using Manhattan Distance
# Importing necessary libraries
from sklearn.metrics.pairwise import manhattan_distances

In [105]:
# Determining manhattan distance
sim_man = manhattan_distances(finaldata)
df_man= pd.DataFrame(sim_man)
df_man.head()

,0,1,2,3,4,5,6,7,8,9,...,240,241,242,243,244,245,246,247,248,249
0,0.000000,3.000000,1.723252,23.732687,2.647756,12.534092,6.969145,2.709929,12.532039,13.815009,...,13.657911,13.792855,11.554098,9.315341,7.076584,11.848643,214.038536,213.038536,213.038536,215.038536
1,3.000000,0.000000,4.723252,26.732687,5.647756,9.534092,3.969145,5.709929,9.532039,10.815009,...,14.657911,14.792855,12.554098,10.315341,8.076584,12.848643,213.038536,214.038536,214.038536,212.038536
2,1.723252,4.723252,0.000000,25.455939,1.075496,14.257344,8.692397,1.013323,14.255291,15.538261,...,13.883751,14.018695,11.779938,9.541181,7.302424,13.571895,213.812696,212.812696,212.812696,214.812696
3,23.732687,26.732687,25.455939,0.000000,26.380442,18.279022,24.763541,26.442616,18.290193,25.903325,...,29.850973,29.835642,26.616261,23.396880,20.177499,19.570682,228.358174,227.358174,227.358174,229.358174
4,2.647756,5.647756,1.075496,26.380442,0.000000,15.181848,9.616901,0.062173,15.179795,16.462765,...,14.867625,15.002568,12.763811,10.525054,8.286297,12.496399,214.828822,213.828822,213.828822,215.828822


In [106]:
# Determining the similar restaurents to the given restaurent
sim_man_rest = df_man.iloc[item].values
print("Manhattan Distance - Restaurent Similarities:\n",sim_man_rest)

Manhattan Distance - Restaurent Similarities:
 [9.21586061e+00 1.22158606e+01 9.44170040e+00 2.35462829e+01
 1.04255742e+01 1.31752448e+01 1.19072077e+01 1.04338548e+01
 1.31864162e+01 2.07995477e+01 1.36606716e+01 2.65518759e+01
 1.27343002e+01 5.57695603e+00 1.78882123e+01 1.49551644e+01
 2.18907551e+01 1.01700998e+01 1.14448458e+01 1.31857230e+01
 2.00055873e+00 1.14472313e+01 1.56970527e+01 1.79468740e+01
 1.40799224e+01 1.32129708e+01 7.00432440e+00 1.68760032e+01
 1.21591273e+01 8.83498340e+00 1.24095819e+01 1.16634325e+01
 1.15482709e+01 1.13879421e+01 1.09216888e+01 1.34548483e+01
 1.01871825e+01 1.22533842e+01 1.14119294e+01 9.44169590e+00
 1.31814759e+01 8.90460760e+00 1.35975897e+01 8.04755370e+00
 6.54709020e+00 1.98307438e+01 1.16809898e+01 1.19035656e+01
 1.14115663e+01 1.68887480e+01 1.21495618e+01 1.34582602e+01
 1.27508003e+01 1.01605919e+01 1.40523728e+01 9.91478282e+00
 1.78059644e+01 1.34472215e+01 2.08063141e+01 9.76302131e+00
 7.87282436e+00 1.68760300e-01 1.34630

In [107]:
# Top-3 approach for recommendation of 3 similar restaurents
# Displaying index of first 3 similar restaurents with restaurent index 132
sim_man_idx = np.argsort(sim_man_rest)[0:3]
print("Manhattan Distance - Index of similar restaurents:\n",sim_man_idx)

Manhattan Distance - Index of similar restaurents:
 [ 77 101  61]


In [108]:
# Distances with the 3 similar restaurents
for i in range(3):
    print("Cosine Distance:",sim_man_rest[i])
    print("Restaurant Name:", fooddata.iloc[sim_man_idx[i]].Restaurant)
    print("Details:", fooddata.iloc[sim_man_idx[i]])
    print("-----------------------------------------------")

Cosine Distance: 9.21586060586937
Restaurant Name: Hotel Abode
Details: Restaurant              Hotel Abode
City                      Hyderabad
Cuisine       Chinese, North Indian
Name: 77, dtype: object
-----------------------------------------------
Cosine Distance: 12.21586060586937
Restaurant Name: Taj Vivanta
Details: Restaurant     Taj Vivanta
City             Hyderabad
Cuisine       North Indian
Name: 101, dtype: object
-----------------------------------------------
Cosine Distance: 9.441700400000002
Restaurant Name: Hotel Nizampet
Details: Restaurant    Hotel Nizampet
City               Hyderabad
Cuisine         North Indian
Name: 61, dtype: object
-----------------------------------------------


In [109]:
# Distances with the 3 similar restaurents
for i in range(3):
    print('Distance  with restaurent at index',sim_man_idx[i],'is:',sim_man_rest[sim_man_idx[i]])
    print("Details of restaurent:\n",fooddata.values[sim_man_idx[i],:])

Distance  with restaurent at index 77 is: 0.0
Details of restaurent:
 ['Hotel Abode' 'Hyderabad' 'Chinese, North Indian']
Distance  with restaurent at index 101 is: 0.04245719999999409
Details of restaurent:
 ['Taj Vivanta' 'Hyderabad' 'North Indian']
Distance  with restaurent at index 61 is: 0.16876029999999176
Details of restaurent:
 ['Hotel Nizampet' 'Hyderabad' 'North Indian']


In [110]:
# Value-based approach for recommendation of similar restaurents
value_idx = np.where(sim_man_rest<2)
print("Number of restaurents <2 value:",len(sum(value_idx)))
print("Index of similar restaurents based on value:\n",value_idx)

Number of restaurents <2 value: 9
Index of similar restaurents based on value:
 (array([ 61,  77,  94, 100, 101, 135, 199, 205, 234]),)
